In [1]:
from brightway2 import *
from bw2io.strategies import *

In [2]:
projects.current = "US LCI"

In [5]:
bw2setup()

Creating default biosphere

Applying strategy: drop_unspecified_subcategories

Writing activities to SQLite3 database:
0%                          100%
[##############################] | ETA[sec]: 0.000 
Total time elapsed: 0.638 sec



Title: Writing activities to SQLite3 database:
  Started: 08/23/2015 09:42:54
  Finished: 08/23/2015 09:42:55
  Total time elapsed: 0.638 sec
  CPU %: 90.900000
  Memory %: 1.392651
Created database: biosphere3
Creating default LCIA methods

Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Wrote 692 LCIA methods with 170915 characterization factors
Creating core data migrations



In [3]:
sp = SingleOutputEcospold1Importer(
    "/Users/cmutel/Documents/LCA Documents/US LCI database/2014", 
    "US LCI"
)

Extracting ecospold1 files:
0%                          100%
[##############################] | ETA[sec]: 0.000 | Item ID: /Users/cmutel/D
Total time elapsed: 1.664 sec


Title: Extracting ecospold1 files:
  Started: 08/23/2015 10:09:06
  Finished: 08/23/2015 10:09:08
  Total time elapsed: 1.664 sec
  CPU %: 100.200000
  Memory %: 1.124382
Extracted 700 datasets in 1.67 seconds


In [4]:
sp.apply_strategies()

Applying strategy: assign_only_product_as_production
Applying strategy: clean_integer_codes
Applying strategy: drop_unspecified_subcategories
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: link_technosphere_by_activity_hash


The US LCI has "dummy" processes - links to activities which are real inputs, but which aren't modeled in the database. We need to add these dummy processes as real activities (even if they don't have any inputs themselves).

In [5]:
sp.apply_strategy(special.add_dummy_processes)

Applying strategy: add_dummy_processes


In [6]:
sp.migrate("biosphere-2-3-names")
sp.migrate("biosphere-2-3-categories")
sp.migrate("us-lci")

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges


We are now ready to start internally linking the database.

We call the `match_database` method with two arguments. The first is `None`, i.e. we are not linking against another database, but only doing internal linking. Because the US LCI doesn't use categories in exchange definitions consistently, we also `ignore_categories`.

In [7]:
sp.match_database(None, ignore_categories=True)

Applying strategy: link_technosphere_based_on_name_unit_location


After the above command, you should see an error - it looks like there are two datasets which are the same. Maybe two different versions, or a renaming where both names where published? The only solution is to choose one, and delete or rename the other.

In [18]:
duplicates = [x for x in sp.data if x['name'] == 'Harvesting, fresh fruit bunch, at farm']

for key in duplicates[0]:
    if key == 'exchanges':
        continue
    if duplicates[0][key] == duplicates[1][key]:
        print("Identical for key {}\n".format(key))
    else:
        print(key)
        print(duplicates[0][key])
        print(duplicates[1][key])

Identical for key production amount

code
9d0df3caf9bc7bb54afb162bd20da096
bc7fb2fb585b3565ca215412d8871cd3
Identical for key location

Identical for key database

categories
['Crop Production']
['Crop Production', 'Other Noncitrus Fruit Farming']
Identical for key type

Identical for key name

Identical for key unit

filename
/Users/cmutel/Documents/LCA Documents/US LCI database/2014/Fresh fruit bunches.xml
/Users/cmutel/Documents/LCA Documents/US LCI database/2014/Harvesting, fresh fruit bunch, at farm.xml
comment
Assumes amount harvested is directly input to palm kernel production with no losses due to land-applied or animal feed wastes. Complete inventory data and metadata are available in full in the final report and appendices, Cradle-to-Gate Life Cycle Inventory of Nine Plastic Resins and Four Polyurethane Precursors. This report has been extensively reviewed within Franklin Associates and has undergone partial critical review by ACC Plastics Division members and is available at

We have done the internal linking that we can - now we need to link the biosphere flows. This looks complicated, but is just a fancy way of linking the biosphere flows by their names, units, and categories.

In [8]:
import functools
f = functools.partial(link_iterable_by_fields,
    other=Database(config.biosphere),
    kind='biosphere'
)
sp.apply_strategy(f)

Applying strategy: link_iterable_by_fields


Let's see how far we have got:

In [9]:
sp.statistics()

957 datasets
31015 exchanges
11151 unlinked exchanges
  Type biosphere: 1233 unique unlinked exchanges
  Type production: 281 unique unlinked exchanges
  Type substitution: 1 unique unlinked exchanges
  Type technosphere: 508 unique unlinked exchanges


(957, 31015, 11151)

Not great. Let's export lists of what we have, and what we can link:

In [11]:
sp.write_excel(only_unlinked=True)

Wrote matching file to:
/Users/cmutel/Library/Application Support/Brightway3/US-LCI.dc95923157ce5b74345603ecff24cb4d/export/db-matching-US-LCI-unlinked.xlsx


In [12]:
sp.write_excel(only_names=True)

Wrote matching file to:
/Users/cmutel/Library/Application Support/Brightway3/US-LCI.dc95923157ce5b74345603ecff24cb4d/export/db-matching-US-LCI-names.xlsx


In [13]:
sp.write_excel()

Wrote matching file to:
/Users/cmutel/Library/Application Support/Brightway3/US-LCI.dc95923157ce5b74345603ecff24cb4d/export/db-matching-US-LCI.xlsx
